# Hands-on: Run the AIFS


This notebook describes how to run inference of the AIFS from an existing checkpoint. 


**Resources:**

[anemoi-inference](https://anemoi-inference.readthedocs.io/en/latest/apis/level1.html) package.

## Install Required Packages and Imports


Fastest installation with `uv`. Get `uv` by running: 
```bash
curl -LsSf https://astral.sh/uv/install.sh | sh

```
Create environment and install required packages:
```bash
uv venv --python=3.12 
source .venv/bin/activate
uv pip install anemoi-training anemoi-models anemoi-inference anemoi-datasets torch torchvision
```

We have already done that for you on the EWC, so you are good to go! 

In [ ]:
from pathlib import Path
import subprocess
import torch
import warnings
import copy
warnings.filterwarnings("ignore", category=UserWarning)


In [ ]:
from anemoi.inference.checkpoint import Checkpoint
from anemoi.inference.runners.simple import SimpleRunner
from anemoi.inference.outputs.printer import print_state

from helpers import find_nearest_point_index

## Investigate the checkpoint

For that we first download the checkpoint:

In [ ]:
url = "https://object-store.os-api.cci1.ecmwf.int/ml-tests/test-data/samples/training-course/inference-aifs-o96.ckpt"
ckpt_file = "../../checkpoints/inference-aifs-o96.ckpt"

# Create the output directory 
Path(ckpt_file).parent.mkdir(parents=True, exist_ok=True)

# Download the checkpoint
if not Path(ckpt_file).exists():
    process = subprocess.run(
        ["wget", url, "-O", ckpt_file],
        capture_output=True,
        text=True
    )
    print(process.stdout)
    print(process.stderr)
else:
    print(f"Checkpoint already exists in {ckpt_file}")


The checkpoint contains all the relevant information about the model and the required input.

In [ ]:
print("Loading checkpoint...")
ckpt = Checkpoint(ckpt_file)

# Variables used during training:
print("Number of variables:", len(ckpt.variable_categories()))
print(f"Diagnostic variables: {ckpt.diagnostic_variables}")
print(f"Prognostic variables: {ckpt.prognostic_variables}")

The model architecture

In [ ]:
model = torch.load(ckpt.path, weights_only=False)
model

In [ ]:
print(f"Number of weights: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

## Get initial conditions


Running the model for inference requires initial conditions. We retrieve Initial Conditions from ECMWF Open Data.




In [ ]:
import datetime
from collections import defaultdict
from pathlib import Path

import numpy as np
import earthkit.data as ekd
import earthkit.regrid as ekr

from ecmwf.opendata import Client as OpendataClient
from helpers import load_saved_state, save_state

In [ ]:
R = 6371.0  # Earth radius in km
GRID_RESOLUTION = "O96"
PARAM_SFC = ["10u", "10v", "2d", "2t", "msl", "skt", "sp", "tcw", "lsm", "z", "slor", "sdor"]
PARAM_PL = ["gh", "t", "u", "v", "w", "q"]
LEVELS = [1000, 925, 850, 700, 600, 500, 400, 300, 250, 200, 150, 100, 50]

DATE = OpendataClient().latest()
print("Initial date is", DATE)

In [ ]:
def get_open_data(param, levelist=[], reference_date=DATE):
    fields = defaultdict(list)
    # Get the data for the current date and the previous date
    for date in [reference_date - datetime.timedelta(hours=6), reference_date]:
        data = ekd.from_source("ecmwf-open-data", date=date, param=param, levelist=levelist)
        for f in data:
            # Open data is between -180 and 180, we need to shift it to 0-360
            assert f.to_numpy().shape == (721, 1440)
            values = np.roll(f.to_numpy(), -f.shape[1] // 2, axis=1)
            # Interpolate the data to from 0.25 to grid
            values = ekr.interpolate(values, {"grid": (0.25, 0.25)}, {"grid": GRID_RESOLUTION})
            # Add the values to the list
            name = f"{f.metadata('param')}_{f.metadata('levelist')}" if levelist else f.metadata("param")
            fields[name].append(values)

    # Create a single matrix for each parameter
    for param, values in fields.items():
        fields[param] = np.stack(values)

    return fields

Here we make a single forecast, from the control initial condition of the ensemble. In operations, this model is run 51 times, once from the control and 50 more times from slightly different initial conditions. 

To get different ensemble initial conditions from the ifs pertubation, set `number` to an ensemble number, between 1-50.


In [ ]:
file = Path(f"inputstate-{GRID_RESOLUTION}-{DATE.strftime('%Y%m%d_%H')}.npz")
if file.exists():
    input_state = load_saved_state(file)
    print(f"Input state loaded from file {file}")
else:
    fields = {}
    # Surface fields
    fields.update(get_open_data(param=PARAM_SFC, reference_date=DATE))
    # Pressure levels fields
    fields.update(get_open_data(param=PARAM_PL, levelist=LEVELS, reference_date=DATE))

    # Convert geopotential height into geopotential
    for level in LEVELS:
        gh = fields.pop(f"gh_{level}")
        fields[f"z_{level}"] = gh * 9.80665
    
    input_state = dict(date=DATE, fields=fields)

    print(f"State created for date {DATE}")
    save_state(input_state, file)

    print(f"State saved to file {file}")

## Run inference from initial conditions

In [ ]:
runner = SimpleRunner(ckpt_file, device="cuda")

In [ ]:
LEAD_TIME = 48

### Run the forecast



In [ ]:
ens_members = 1 # This will become important tomorrow
forecast_states = [[] for nn in range(ens_members)]
for nn in range(ens_members):
    for step, state in enumerate(runner.run(input_state=input_state, lead_time=LEAD_TIME)):
        print(f"Step {step}: {state['date']} (lead time: {state['step']})")
        print(f"  Available fields: {list(state['fields'].keys())}")
        # Store the state if you want to keep it
        forecast_states[nn].append(copy.deepcopy(state))

print(f"Forecast completed! Generated {len(forecast_states)} time steps.")

In [ ]:
print(f"Predicted dates: {list(forecast_states[0][x]['date'].strftime('%Y-%m-%dT%H') for x in range(len(forecast_states[0])))}")

# 4. Inspect the generated forecast

### Plot a field

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.tri as tri

In [ ]:
def fix(lons):
    # Shift the longitudes from 0-360 to -180-180
    return np.where(lons > 180, lons - 360, lons)

# pick a state of the first ensemble members
variable = "10u"
state = forecast_states[0][-1]
date = state['date']
latitudes = state["latitudes"]
longitudes = fix(state["longitudes"])
values = state["fields"][variable]

In [ ]:
fig, ax = plt.subplots(figsize=(11, 6), subplot_kw={"projection": ccrs.PlateCarree()})
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=":")

triangulation = tri.Triangulation(fix(longitudes), latitudes)

contour=ax.tricontourf(triangulation, values, levels=20, transform=ccrs.PlateCarree(), cmap="RdBu")
cbar = fig.colorbar(contour, ax=ax, orientation="vertical", shrink=0.7, label="10u")

plt.title("{} at {}".format(variable, state["date"]))
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(11, 6), subplot_kw={"projection": ccrs.PlateCarree()})
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=":")

# Limit the map to Europe
ax.set_extent([-25, 45, 34, 72], crs=ccrs.PlateCarree())

triangulation = tri.Triangulation(fix(longitudes), latitudes)

contour=ax.tricontourf(triangulation, values, levels=20, transform=ccrs.PlateCarree(), cmap="RdBu")
cbar = fig.colorbar(contour, ax=ax, orientation="vertical", shrink=0.7, label="10u")

plt.title("{} at {}".format(variable, state["date"]))
plt.show()

### Task 4.1: Plot the 2m temperature forecasts for the weekend?

The forecast is already calculated. Just make sure to select the correct state from the list and select the wanted parameter. The legend must be edited manually. Feel free to convert from Kelvin to Celcius.

### Plot a forecast time-series 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

lat_target = 51.45
lon_target = -0.98
idx_nearest = find_nearest_point_index(latitudes, longitudes, lat_target, lon_target)

lead_times = range(len(forecast_states[0]))
n_members = 1  # This will become important tomorrow

# Extract field for all members and times
values = np.array([
    [forecast_states[m][t]["fields"][variable][idx_nearest] for t in lead_times]
    for m in range(n_members)
])

# Time axis
times = [forecast_states[0][t]["date"] for t in lead_times]
times = pd.to_datetime(times)

# Plot
plt.figure(figsize=(10, 5))

for m in range(n_members):
    plt.plot(times, values[m, :], linewidth=2, label=f"Member {m+1}")

plt.title(f"{variable} forecast at lat={latitudes[idx_nearest]}, lon={longitudes[idx_nearest]}")
plt.xlabel("Time")
plt.ylabel(variable)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### Task 4.2: Plot a 2m temperature forecast for the location of your city?

We have provided an example for the Reading, UK. Change to the location of the city you live in and and select the wanted parameter. 

Extra task: Try to calculate the forecast for more than 48 hours to have an idea of how warm it is going to be next week.

### Task 5: Explore out of sample data in the initial conditions

We have provided a way to get data which matches what the model was trained for, explore what happens if the data is incorrect 